In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 64, kernel_size=2, stride=1, padding=0),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 64
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AlinaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.action_head = nn.Linear(features_dim, action_space.n)
        self.value_head = nn.Linear(features_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C17']))
    env = PovOnlyWrapper(env)
    env = IgluActionWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C17 pretrained"
                  }
              }

        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


Trial name,status,loc
PPO_my_env_421d3_00000,PENDING,


2021-09-18 11:04:56,824	INFO wandb.py:170 -- Already logged into W&B.
2021-09-18 11:04:56,836	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
(pid=58357) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=58357)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=58357) 2021-09-18 11:05:00,414	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=58357) 2021-09-18 11:05:00,414	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=58353) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=58353)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=58357) 2021-09-18 11:05:06,477	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=58353) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=58353)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=58353) /root/miniconda/envs/py37/lib/python3.7/

Result for PPO_my_env_421d3_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2021-09-18_11-06-54
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: e1746907352f4fe585fcb0799b3f64f3
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 0.2018264439370897
          entropy_coeff: 0.009999999999999998
          kl: 0.0011697852865390403
          policy_loss: -0.19659400859640705
          total_loss: -0.19176375745899146
          vf_explained_var: 0.24490948021411896
          vf_loss: 0.00661455605748213
    num_agent_steps_sampled: 1000
    num_agent_steps_trained: 1000
    num_steps_sampled: 1000
    num_steps_trained: 1000
  iterations_since_restore: 1
  node_ip: 192.168.3.5

...

Result for PPO_my_env_421d3_00000:
  agent_timesteps_total: 38000
  custom_metrics: {}
  date: 2021-09-18_11-13-24
  done: false
  episode_len_mean: 996.8947368421053
  episode_media: {}
  episode_reward_max: 3.0
  episode_reward_mean: -0.05263157894736842
  episode_reward_min: -5.0
  episodes_this_iter: 1
  episodes_total: 38
  experiment_id: e1746907352f4fe585fcb0799b3f64f3
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.00012359619140625
          cur_lr: 5.000000000000001e-05
          entropy: 2.2625883473290336
          entropy_coeff: 0.009999999999999998
          kl: 0.01029669067981271
          policy_loss: -0.11208353485498164
          total_loss: -0.1282469150920709
          vf_explained_var: 0.20722801983356476
          vf_loss: 0.0064612270746794015
    num_agent_steps_sampled: 38000
    num_agent_steps_trained: 38000
    num_steps_sampled: 38000
    num_steps_trained: 38000
  iterations_since

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_421d3_00000,RUNNING,192.168.3.5:58357,38,496.868,38000,-0.0526316,3,-5,996.895


2021-09-18 11:13:26,783	WARNING util.py:164 -- The `on_step_begin` operation took 1.497 s, which may be a performance bottleneck.


Result for PPO_my_env_421d3_00000:
  agent_timesteps_total: 39000
  custom_metrics: {}
  date: 2021-09-18_11-13-37
  done: false
  episode_len_mean: 996.974358974359
  episode_media: {}
  episode_reward_max: 3.0
  episode_reward_mean: -0.05128205128205128
  episode_reward_min: -5.0
  episodes_this_iter: 1
  episodes_total: 39
  experiment_id: e1746907352f4fe585fcb0799b3f64f3
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.00012359619140625
          cur_lr: 5.000000000000001e-05
          entropy: 2.2796993997361925
          entropy_coeff: 0.009999999999999998
          kl: 0.01315724665048656
          policy_loss: 0.05450283686319987
          total_loss: 0.03510063687960307
          vf_explained_var: 0.12637978792190552
          vf_loss: 0.0033931700862012805
    num_agent_steps_sampled: 39000
    num_agent_steps_trained: 39000
    num_steps_sampled: 39000
    num_steps_trained: 39000
  iterations_since_r

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_421d3_00000,RUNNING,192.168.3.5:58357,39,509.403,39000,-0.0512821,3,-5,996.974
